In [2]:
import torch
import torch.nn.functional as F
import random
import os
import numpy as np
import torch.nn as nn
import torchvision.transforms as T
import dataset
import Model
import train
import config as cf
import prepare
import remove
import utils
import copy
import time

In [3]:
utils.random_setting(0)

In [4]:
label = 0
UNLEARN_LABEL = [label]
OUTPUT_LABEL = [i for i in range(cf.CLASS_COUNT) if i != label]
UNLEARNED_DISTRIBUTION = [0 for i in range(cf.CLASS_COUNT)] 
UNLEARNED_DISTRIBUTION[label] = 1

In [5]:
data = dataset.data_construction(cf.DATASET)        
loaders = data.construct_data(cf.DATATYPE, UNLEARNED_DISTRIBUTION)

Train data length:  60000
Test data length:  10000
[5923. 6742. 5958. 6131. 5842. 5421. 5918. 6265. 5851. 5949.]
[5923.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [7]:
if cf.DATASET == 'Digit' or cf.DATASET == 'Fashion':
    trained_model = Model.CNN()
elif cf.DATASET == 'CIFAR10' or cf.DATASET == 'CIFAR100':
    trained_model = Model.ResNet()

In [8]:
train.train(trained_model, loaders['train'])
            
new_model = copy.deepcopy(trained_model)

In [10]:
time_0 = time.time()
close_dataset = prepare.counterfactual_generation(trained_model, loaders['unlearn'], loaders['sample'])
close_loader = torch.utils.data.DataLoader(close_dataset, batch_size=32, shuffle=True, num_workers=3)

In [11]:
remove.causal_effect_removel(trained_model, new_model, close_loader, loaders['sample'])

time_taken = time.time() - time_0

In [12]:

add_model = copy.deepcopy(new_model)

attack_model = train.train_attack_model(add_model, loaders['sample'], loaders['test'])    
acc_re= train.attack(add_model, attack_model, loaders['unlearn'], loaders['test'])
acc_re_tr = train.test(add_model, loaders['source'])
acc_fr_tr = train.test(add_model, loaders['unlearn'])
acc_ts = train.test(add_model, loaders['test'])
acc_re_ts = train.test(add_model, loaders['test'], OUTPUT_LABEL)
acc_fr_ts = train.test(add_model, loaders['test'], UNLEARN_LABEL)

4000
4000
MIA Attacker accuracy = 0.9157
MIA Attacker accuracy = 0.2319
The accuracy of the all classes is:  99.81138006916063
The accuracy of the all classes is:  0.0
The accuracy of the all classes is:  89.54
The accuracy of the selected classes  [1, 2, 3, 4, 5, 6, 7, 8, 9] is:  99.26829268292683
The accuracy of the selected classes  [0] is:  0.0


In [13]:
print('Unlearn remaining training acc is ', acc_re_tr)
print('Unlearn forgetting training acc is ', acc_fr_tr)
print('Unlearn test acc is ', acc_ts)
print('Unlearn remaining test acc is ', acc_re_ts)
print('Unlearn forgetting test acc is ', acc_fr_ts)
print('Attack Success Rate is ', acc_re)
print('Algorithm Time is ', time_taken)

Unlearn remaining training acc is  99.81138006916063
Unlearn forgetting training acc is  0.0
Unlearn test acc is  89.54
Unlearn remaining test acc is  99.26829268292683
Unlearn forgetting test acc is  0.0
Attack Success Rate is  0.23191578947368421
Algorithm Time is  9.998535633087158
